# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Häberle  ->  M. Häberle  |  ['M. Häberle']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
A. Dumont  ->  A. Dumont  |  ['A. Dumont']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
P. Eitner  ->  P. Eitner  |  ['P. Eitner']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
R. Hoppe  ->  R. Hoppe  |  ['R. Hoppe']
J. Klevas  ->  J. Klevas  |  ['J. Klevas']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 55 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/
        2405.06015
      
Retrieving document from  https://arxiv.org/e-print/
        2405.06020
      
Retrieving document from  https://arxiv.org/e-print/
        2405.06047
      
Retrieving document from  https://arxiv.org/e-print/
        2405.06338
      
Retrieving document from  https://arxiv.org/e-print/
        2405.06470
      


/tmp/ipykernel_2217/3009462391.py:49: LatexWarning: 
        2405.06015
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.06015' (found at least '\n')
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2217/3009462391.py:49: LatexWarning: 
        2405.06020
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.06020' (found at least '\n')
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2217/3009462391.py:49: LatexWarning: 
        2405.06047
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.06047' (found at least '\n')
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
/tmp/ipykernel_2217/3009462391.py:49: LatexWarning: 
        2405.06338
       did not run properly
URL can't contain control characters. '/e-print/\n        2405.06338' (found at least '\n')
  wa

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.06015
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.06015
      ) | **Fast-moving stars around an intermediate-mass black hole in Omega Centauri**  |
|| <mark>M. Häberle</mark>, et al. -- incl., <mark>N. Neumayer</mark>, <mark>A. Dumont</mark>, <mark>C. Clontz</mark> |
|*Appeared on*| *2024-05-13*|
|*Comments*| *33 pages, 11 figures and 2 tables. Accepted for publication in Nature. It is embargoed for discussion in the press until formal publication in Nature. Several press releases are pending*|
|**Abstract**|            Black holes have been found over a wide range of masses, from stellar remnants with masses of 5-150 solar masses (Msun), to those found at the centers of galaxies with $M>10^5$ Msun. However, only a few debated candidate black holes exist between 150 and $10^5$ Msun. Determining the population of these intermediate-mass black holes is an important step towards understanding supermassive black hole formation in the early universe. Several studies have claimed the detection of a central black hole in $\omega$ Centauri, the Milky Way's most massive globular cluster. However, these studies have been questioned due to the possible mass contribution of stellar mass black holes, their sensitivity to the cluster center, and the lack of fast-moving stars above the escape velocity. Here we report observations of seven fast-moving stars in the central 3 arcseconds (0.08 pc) of $\omega$ Centauri. The velocities of the fast-moving stars are significantly higher than the expected central escape velocity of the star cluster, so their presence can only be explained by being bound to a massive black hole. From the velocities alone, we can infer a firm lower limit of the black hole mass of $\sim$8,200 Msun, making this a compelling candidate for an intermediate-mass black hole in the local universe.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.06015' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.06020
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.06020
      ) | **Wide post-common envelope binaries from Gaia: orbit validation and formation models**  |
|| N. Yamaguchi, et al. -- incl., <mark>K. El-Badry</mark>, <mark>R. Andrae</mark> |
|*Appeared on*| *2024-05-13*|
|*Comments*| *36 pages, 22 figures, submitted to PASP*|
|**Abstract**|            Astrometry from {\it Gaia} DR3 has enabled the discovery of a sample of 3000+ binaries containing white dwarfs (WD) and main-sequence (MS) stars in relatively wide orbits, with orbital periods $P_{\rm orb} = (100-1000)$ d. This population was not predicted by binary population synthesis models before {\it Gaia} and -- if the {\it Gaia} orbits are robust -- likely requires very efficient envelope ejection during common envelope evolution (CEE). To assess the reliability of the {\it Gaia} solutions, we measured multi-epoch radial velocities (RVs) of 31 WD+MS binary candidates with $P_{\rm orb} = (40-300)$ d and \texttt{AstroSpectroSB1} orbital solutions. We jointly fit the RVs and astrometry, allowing us to validate the {\it Gaia} solutions and tighten constraints on component masses. We find a high success rate for the {\it Gaia} solutions, with only 2 out of the 31 systems showing significant discrepancies between their {\it Gaia} orbital solutions and our RVs. Joint fitting of RVs and astrometry allows us to directly constrain the secondary-to-primary flux ratio $\mathcal{S}$, and we find $\mathcal{S}\lesssim 0.02$ for most objects, confirming the companions are indeed WDs. We tighten constraints on the binaries' eccentricities, finding a median $e\approx 0.1$. These eccentricities are much lower than those of normal MS+MS binaries at similar periods, but much higher than predicted for binaries formed via stable mass transfer. We present MESA single and binary evolution models to explore how the binaries may have formed. The orbits of most binaries in the sample can be produced through CEE that begins when the WD progenitor is an AGB star, corresponding to initial separations of $2-5$ au. Roughly 50\% of all post-common envelope binaries are predicted to have first interacted on the AGB, ending up in wide orbits like these systems.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.06020' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.06047
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.06047
      ) | **Euclid preparation. Sensitivity to neutrino parameters**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-05-13*|
|*Comments*| *48 pages, 33 figures*|
|**Abstract**|            The Euclid mission of the European Space Agency will deliver weak gravitational lensing and galaxy clustering surveys that can be used to constrain the standard cosmological model and extensions thereof. We present forecasts from the combination of these surveys on the sensitivity to cosmological parameters including the summed neutrino mass $M_\nu$ and the effective number of relativistic species $N_{\rm eff}$ in the standard $\Lambda$CDM scenario and in a scenario with dynamical dark energy ($w_0 w_a$CDM). We compare the accuracy of different algorithms predicting the nonlinear matter power spectrum for such models. We then validate several pipelines for Fisher matrix and MCMC forecasts, using different theory codes, algorithms for numerical derivatives, and assumptions concerning the non-linear cut-off scale. The Euclid primary probes alone will reach a sensitivity of $\sigma(M_\nu)=$56meV in the $\Lambda$CDM+$M_\nu$ model, whereas the combination with CMB data from Planck is expected to achieve $\sigma(M_\nu)=$23meV and raise the evidence for a non-zero neutrino mass to at least the $2.6\sigma$ level. This can be pushed to a $4\sigma$ detection if future CMB data from LiteBIRD and CMB Stage-IV are included. In combination with Planck, Euclid will also deliver tight constraints on $\Delta N_{\rm eff}< 0.144$ (95%CL) in the $\Lambda$CDM+$M_\nu$+$N_{\rm eff}$ model, or $\Delta N_{\rm eff}< 0.063$ when future CMB data are included. When floating $(w_0, w_a)$, we find that the sensitivity to $N_{\rm eff}$ remains stable, while that to $M_\nu$ degrades at most by a factor 2. This work illustrates the complementarity between the Euclid spectroscopic and imaging/photometric surveys and between Euclid and CMB constraints. Euclid will have a great potential for measuring the neutrino mass and excluding well-motivated scenarios with additional relativistic particles.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.06047' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.06338
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.06338
      ) | **M3DIS - A grid of 3D radiation-hydrodynamics stellar atmosphere models for stellar surveys**  |
|| <mark>P. Eitner</mark>, et al. -- incl., <mark>M. Bergemann</mark>, <mark>R. Hoppe</mark>, <mark>J. Klevas</mark> |
|*Appeared on*| *2024-05-13*|
|*Comments*| *Accepted by A&A. 20 pages, 16 figures*|
|**Abstract**|            Large-scale stellar surveys, such as SDSS-V, 4MOST, WEAVE, and PLATO, require accurate atmospheric models and synthetic spectra of stars for accurate analyses of fundamental stellar parameters and chemical abundances. The primary goal of our work is to develop a new approach to solve radiation-hydrodynamics (RHD) and generate model stellar spectra in a self-consistent and highly efficient framework. We build upon the Copenhagen legacy RHD code, the MULTI3D non-local thermodynamic equilibrium (NLTE) code, and the DISPATCH high-performance framework. The new approach allows us to calculate 3D RHD models of stellar atmospheres on timescales of a few thousand CPU hours and to perform subsequent spectrum synthesis in local thermodynamic equilibrium (LTE) or NLTE for the desired physical conditions within the parameter space of FGK-type stars. We compare the 3D RHD solar model with other available models and validate its performance against solar observations, including the centre-to-limb variation of intensities and key solar diagnostic lines of H and Fe. We show that the performance of the new code allows to overcome the main bottleneck in 3D NLTE spectroscopy and enables calculations of multi-dimensional grids of synthetic stellar observables for comparison with modern astronomical observations.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.06338' (found at least '\n')</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-
        arXiv:2405.06470
      -b31b1b.svg)](https://arxiv.org/abs/
        arXiv:2405.06470
      ) | **Solar fusion III: New data and theory for hydrogen-burning stars**  |
|| B. Acharya, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-05-13*|
|*Comments*| *85 pages, 15 figures. To be submitted to Reviews of Modern Physics*|
|**Abstract**|            In stars that lie on the main sequence in the Hertzsprung Russel diagram, like our sun, hydrogen is fused to helium in a number of nuclear reaction chains and series, such as the proton-proton chain and the carbon-nitrogen-oxygen cycles. Precisely determined thermonuclear rates of these reactions lie at the foundation of the standard solar model. This review, the third decadal evaluation of the nuclear physics of hydrogen-burning stars, is motivated by the great advances made in recent years by solar neutrino observatories, putting experimental knowledge of the proton-proton chain neutrino fluxes in the few-percent precision range. The basis of the review is a one-week community meeting held in July 2022 in Berkeley, California, and many subsequent digital meetings and exchanges. Each of the relevant reactions of solar and quiescent stellar hydrogen burning is reviewed here, from both theoretical and experimental perspectives. Recommendations for the state of the art of the astrophysical S-factor and its uncertainty are formulated for each of them. Several other topics of paramount importance for the solar model are reviewed, as well: recent and future neutrino experiments, electron screening, radiative opacities, and current and upcoming experimental facilities. In addition to reaction-specific recommendations, also general recommendations are formed.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error URL can't contain control characters. '/e-print/\n        2405.06470' (found at least '\n')</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

414  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
